In [5]:
with open("shipping_QA_data.txt", encoding="utf-8") as f:
    shipping_customer_service = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [6]:
from langchain.text_splitter import CharacterTextSplitter

In [7]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [8]:
docs = text_splitter.create_documents([shipping_customer_service])

Created a chunk of size 103, which is longer than the specified 100
Created a chunk of size 110, which is longer than the specified 100
Created a chunk of size 101, which is longer than the specified 100
Created a chunk of size 105, which is longer than the specified 100
Created a chunk of size 116, which is longer than the specified 100
Created a chunk of size 102, which is longer than the specified 100
Created a chunk of size 107, which is longer than the specified 100
Created a chunk of size 102, which is longer than the specified 100
Created a chunk of size 112, which is longer than the specified 100
Created a chunk of size 104, which is longer than the specified 100
Created a chunk of size 116, which is longer than the specified 100
Created a chunk of size 104, which is longer than the specified 100
Created a chunk of size 154, which is longer than the specified 100
Created a chunk of size 101, which is longer than the specified 100
Created a chunk of size 114, which is longer tha

In [9]:
docs[0]

Document(page_content='[客户问题] 请问你们提供哪些类型的国际运输服务？\n[销售回答] 您好，我们提供全面的国际运输服务，包括海运整箱、拼箱、空运、陆运以及多式联运等，根据您的具体需求，我们可以为您定制最合适的运输方案。')

In [10]:
len(docs)

51

### 使用 Faiss 作为向量数据库，持久化存储 问答对（QA-Pair）

In [11]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())

c:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [12]:
query = "如何追踪我的货物在运输途中的状态"

In [13]:
answer_list = db.similarity_search(query)

In [14]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题] 如何追踪我的货物在运输途中的状态？
[销售回答] 我们提供实时的货物追踪服务，您可以通过我们的官方网站或客服热线查询货物的最新动态。我们的系统会定期更新货物位置信息，确保您随时掌握运输情况。

[客户问题] 你们是否支持货物追踪的短信或邮件通知服务？
[销售回答] 是的，我们提供货物追踪的短信和邮件通知服务，让您随时掌握货物的运输状态。

[客户问题] 你们是否提供运输跟踪软件的接口，以便我们系统集成？
[销售回答] 是的，我们提供运输跟踪软件的接口，方便您与我们的系统进行集成。通过接口，您可以实时获取运输信息，并与您的业务流程进行无缝对接。

[客户问题] 你们能否提供货物的实时照片或视频，以确认货物状态？
[销售回答] 我们尽力提供货物的实时照片或视频服务，但具体可行性取决于运输环节和当前情况。我们会与承运方沟通，尽可能满足您的需求。



In [40]:
db.save_local("shipping_qa_pair")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [16]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [17]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002C03C49A950>, search_kwargs={'k': 3})

In [18]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

c:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[客户问题] 如何追踪我的货物在运输途中的状态？
[销售回答] 我们提供实时的货物追踪服务，您可以通过我们的官方网站或客服热线查询货物的最新动态。我们的系统会定期更新货物位置信息，确保您随时掌握运输情况。

[客户问题] 你们是否支持货物追踪的短信或邮件通知服务？
[销售回答] 是的，我们提供货物追踪的短信和邮件通知服务，让您随时掌握货物的运输状态。

[客户问题] 你们是否提供运输跟踪软件的接口，以便我们系统集成？
[销售回答] 是的，我们提供运输跟踪软件的接口，方便您与我们的系统进行集成。通过接口，您可以实时获取运输信息，并与您的业务流程进行无缝对接。



In [19]:
docs = topK_retriever.get_relevant_documents("你们有没有1000万的集装箱啊？")

In [21]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 你们有海外仓储服务吗？
[销售回答] 是的，我们提供海外仓储服务，包括仓储管理、分拣包装、代发货等一站式解决方案。这可以帮助您更好地管理库存，提高物流效率。

[客户问题] 我需要紧急运输一批货物，你们能提供加急服务吗？
[销售回答] 当然可以，我们提供紧急加急服务，会尽快安排运输，确保货物在最短时间内送达。

[客户问题] 我对货物包装有特殊要求，你们能满足吗？
[销售回答] 我们提供专业的货物包装服务，可以根据您的特殊要求定制包装方案。请提供详细的包装要求，我们会安排专业的包装团队进行操作。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [22]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [23]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 如何追踪我的货物在运输途中的状态？
[销售回答] 我们提供实时的货物追踪服务，您可以通过我们的官方网站或客服热线查询货物的最新动态。我们的系统会定期更新货物位置信息，确保您随时掌握运输情况。

[客户问题] 你们是否支持货物追踪的短信或邮件通知服务？
[销售回答] 是的，我们提供货物追踪的短信和邮件通知服务，让您随时掌握货物的运输状态。

[客户问题] 你们是否提供运输跟踪软件的接口，以便我们系统集成？
[销售回答] 是的，我们提供运输跟踪软件的接口，方便您与我们的系统进行集成。通过接口，您可以实时获取运输信息，并与您的业务流程进行无缝对接。

[客户问题] 你们能否提供货物的实时照片或视频，以确认货物状态？
[销售回答] 我们尽力提供货物的实时照片或视频服务，但具体可行性取决于运输环节和当前情况。我们会与承运方沟通，尽可能满足您的需求。



### 提取向量数据库中的`客服回答`

In [24]:
docs = retriever.get_relevant_documents(query)

In [25]:
docs[0].page_content

'[客户问题] 如何追踪我的货物在运输途中的状态？\n[销售回答] 我们提供实时的货物追踪服务，您可以通过我们的官方网站或客服热线查询货物的最新动态。我们的系统会定期更新货物位置信息，确保您随时掌握运输情况。'

In [26]:
docs[0].page_content.split("[销售回答] ")

['[客户问题] 如何追踪我的货物在运输途中的状态？\n',
 '我们提供实时的货物追踪服务，您可以通过我们的官方网站或客服热线查询货物的最新动态。我们的系统会定期更新货物位置信息，确保您随时掌握运输情况。']

In [27]:
ans = docs[0].page_content.split("[销售回答] ")[-1]

In [28]:
ans

'我们提供实时的货物追踪服务，您可以通过我们的官方网站或客服热线查询货物的最新动态。我们的系统会定期更新货物位置信息，确保您随时掌握运输情况。'

#### 尝试各种问题

In [29]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]

    return ans_list

In [30]:
query = "我想快点送达"

print(sales(query))

[]


c:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [31]:
print(sales(query, 0.75))

['当然可以，我们提供紧急加急服务，会尽快安排运输，确保货物在最短时间内送达。', '当然可以，我们提供加急运输服务，包括空运特快、海运加班船等。请提前告知我们您的紧急需求，我们会尽快为您安排。']


In [32]:
query = "价格200万以内"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

c:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


score:0.8 ans: []



c:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(


score:0.75 ans: []

score:0.5 ans: ['是的，我们可以提供货物价值评估服务，帮助您了解货物的市场价值，以便更好地进行保险投保或运输决策。', '是的，我们提供全程门到门服务，包括上门取货、运输、清关、送货上门等一站式服务，让您省心省力。', '我们尽力提供货物的实时照片或视频服务，但具体可行性取决于运输环节和当前情况。我们会与承运方沟通，尽可能满足您的需求。', '当然可以，我们提供专业的报关服务，包括单证准备、海关申报、税费缴纳等。我们会根据您的货物情况提供一站式报关解决方案。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [34]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [36]:
qa_chain({"query": "你们集装箱有200万的货物吗？"})

c:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '你们集装箱有200万的货物吗？',
 'result': '我不是集装箱货物销售或物流服务提供者。如果你正在寻找价值200万的货物或集装箱运输服务，你需要联系专业的货运代理、物流公司或者货物供应商来获取帮助和详细信息。'}

In [37]:
qa_chain({"query": "运力大不大"})

c:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '运力大不大',
 'result': '"运力大不大"这个问题通常是在询问某个运输系统、公司或者服务的运输能力是否充足或者规模是否庞大。不过，因为你没有提供具体的上下文，比如是在问某个物流公司、公共交通系统还是别的什么，所以我无法给出具体的答案。如果你能提供更多的信息，我或许能够给出更准确的回答。'}

In [39]:
print(sales("运力大不大"))

[]


c:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(
